In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
from openpyxl import Workbook

page=1
list = []
for j in range(1,68):
    url = 'https://ifoodie.tw/explore/%E5%8F%B0%E5%8D%97%E5%B8%82/list?page={}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
    }

    res = requests.get(url.format(page), headers=headers)
    html = res.text

    soup = BeautifulSoup(html, 'html.parser')

    titlelist = soup.find_all('a', class_="jsx-2133253768 click-tracker")
    
    for title in titlelist:
        urlshop = "https://ifoodie.tw" + title.get('href')
        reshop = requests.get(urlshop, headers=headers)
        htmlshop = reshop.text
        shupshop = BeautifulSoup(htmlshop, 'html.parser')

        shopname = shupshop.find('a', class_="jsx-307016528")
        list.append(shopname.text)
        shopinfo = shupshop.find_all('span', class_="jsx-1692663080 detail", limit=2)
        # print(titlelist)
        list.append(shopinfo[0].text)
        try:
            if '均消' in shopinfo[1].text:
                list.append(shopinfo[1].text)
            else:
                list.append('無')
        except:
            list.append('無')
        shoprank = shupshop.find('div', class_="jsx-1207467136 text")
        list.append(shoprank.text)
        shoptag = shupshop.find_all('a', class_="jsx-307016528 cat-link")
        tag=[]
        for info in shoptag:
            tag.append(info.text)
        tag.remove('附近')
        if len(tag)<1:
            tag.append('暫無')
        list.append(tag)
        # print ("https://ifoodie.tw"+title.get('href'))
    # print(titleSoupList[0])
    # print(list)
    page+=1
    
step = 5
slicelist = [list[i:i + step] for i in range(0, len(list), step)]
shop_df = pd.DataFrame(slicelist)
shop_df.index = shop_df.index + 1  # 自訂索引值
shop_df.columns = ["店名", "地址", "均消", "店評價","店標籤"]
print(shop_df)

shop_df.to_excel('shopinfo.xlsx')

                        店名                    地址       均消  店評價  \
1                  上海華都小吃城        臺南市中西區民權路二段28號  均消 $200  3.4   
2                  天然酵素臭豆腐       臺南市仁德區中正路二段388號        無  4.2   
3         餃津 JiauJin 煎餃專賣店       臺南市永康區自強路462之1號        無  5.0   
4                Batwo全日餐館          臺南市東區生產路512號        無  4.3   
5             友.友 TomoTomo         臺南市中西區康樂街218號   均消 $60  4.3   
..                     ...                   ...      ...  ...   
983                川囍紅湯串串鍋        臺南市東區中華東路二段71號  均消 $250  4.4   
984               麻坡韓國炭火燒肉           臺南市安平區怡平路3號  均消 $600  4.4   
985                翻滾吧，蛋捲!  臺南市東區林森路二段192巷35弄23號  均消 $260  4.5   
986  uMeal Bistro 台南店-知事官邸            臺南市東區衛民街1號        無  4.8   
987                  紅象鐵板燒          臺南市北區東豐路251號        無  4.6   

                                  店標籤  
1            [台南小吃, 台南合菜, 台南晚餐, 台南午餐]  
2                              [台南晚餐]  
3                        [台南晚餐, 台南午餐]  
4                        [台南晚餐, 台南午餐]  
5      